## Imports

In [ ]:
# json module makes getting and working with JSON response easier
import json
import os

In [ ]:
# Try to make our code both py2 and py3 compatible
# py3
try:  
    import http.client as client
    import urllib.parse as parse
    from urllib.request import urlopen as urlopen
    from urllib.request import Request as request
    from urllib.parse import urlencode as encode
# py2
except ImportError:  
    import httplib as client
    from urllib2 import urlparse as parse
    from urllib2 import urlopen as urlopen
    from urllib2 import Request as request
    from urllib import urlencode as encode

## Simple request

In [ ]:
### make a single generic request and look at the response
URL = "http://sampleserver6.arcgisonline.com/arcgis/rest/services/PhoneIncidents/FeatureServer/0/query"
query = "?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&f=json"

In [ ]:
#Build request
r = request(URL + query)
#Open the URL
jsonResponse = urlopen(r)
#Load the JSON response
jsonOuput = json.loads(jsonResponse.read().decode('UTF-8'))
print(jsonOuput)

In [ ]:
#why JSON?
print("read type: {0}".format(type(jsonResponse.read())))
print("json type: {0}".format(type(jsonOuput)))

In [ ]:
### display the response, do a LEN on the array to count # of features
print (jsonOuput)
#How many features?
len(jsonOuput['features'])

In [ ]:
### Build a helper function so we arent repeating code
def sendReq(URL, query_dict=None, headers=None):
    # Takes a URL and optional dict/headers and sends the request; returns the JSON    

    if query_dict:  #POST
        query_string = encode(query_dict).encode('UTF-8')
    else:  #GET
        query_string = encode('').encode('UTF-8')

    if headers:
        req = request(URL)
        for key, value in headers.iteritems():
            req.add_header(key, value)
    else:
        req = URL

    jsonResponse = urlopen(req, query_string)
    jsonOuput = json.loads(jsonResponse.read().decode('utf-8'))

    return jsonOuput

In [ ]:
### Use helper function
URL = "http://sampleserver6.arcgisonline.com/arcgis/rest/services/PhoneIncidents/FeatureServer/0/query"
query = {'where':'1=1',
         'returnGeometry':'true',
         'returnIdsOnly': 'false',
         'f':'json',
         'token': None         
         }
response = sendReq(URL, query)
print (response)
if "features" in response:
    print ("There are {0} features in the response".format(len(response['features'])))
else:
    print ("No features found, or error returned")

##Requests with arcgis.com feature services

In [ ]:
chiptoleURL = "http://services5.arcgis.com/fef1QOfOwB7ly9yl/arcgis/rest/services/ChiptolePlaces/FeatureServer/0"

In [ ]:
#Get the count from the end point
FSquery = {'where':'1=1',
         'returnCountOnly': 'true',
         'f':'json'
         }
queryURL = chiptoleURL + '/query'
response = sendReq(queryURL, FSquery)
print(response)

In [ ]:
FSquery2 = {'where':'1=1',
         'returnGeometry':'true',
         'returnIdsOnly': 'false',
         'f':'json'         
         }
queryURL2 = chiptoleURL + '/query'
response = sendReq(queryURL2, FSquery2)
print(len(response['features']))

**Need to authenticate to change settings on the service**

In [ ]:
### TOKEN FUNCTION
def getToken(username, password, url):

    tokenURL = url + "/sharing/generateToken"
    if "https" not in tokenURL:
        tokenURL =tokenURL.replace('http', 'https')
    referer = 'http://www.arcgis.com'

    query_dict = {'username':   username,
                  'password':   password,
                  'expiration': str(600),
                  'client' : 'referer',
                  'referer' : referer,
                  'f': 'json'} 

    token = sendReq(tokenURL, query_dict) 

    if "token" not in token:
        print (token['messages'])
    else:
        # Return the token
        return token['token']

In [ ]:
username = "khibmaUC"
password = "kevinUC15"
tokenURL = 'http://www.arcgis.com'
token = getToken(username, password, tokenURL)

In [ ]:
print(token)

In [ ]:
updateQuery = {'updateDefinition':
                {'maxRecordCount':5000},
               'f':'json',
               'async':'false',
               'token':token               
              }
updateURL = "http://services5.arcgis.com/fef1QOfOwB7ly9yl/arcgis/rest/admin/services/ChiptolePlaces/FeatureServer/0/updateDefinition"
responseUpdate = sendReq(updateURL, updateQuery)
print(responseUpdate)


In [ ]:
response2 = sendReq(queryURL2, FSquery2)
print(len(response2['features']))

## Send query to hosted service
## Use response in arcpy (tools)

In [ ]:
import arcpy
bufferResult =  r"c:\temp\ChipBuffer.shp"
arcpy.Exists(bufferResult)

In [ ]:
fs = arcpy.FeatureSet()
fs.load("http://services5.arcgis.com/fef1QOfOwB7ly9yl/arcgis/rest/services/ChiptolePlaces/FeatureServer/0/query?where=1=1&f=json")
arcpy.Buffer_analysis(fs, r"c:\temp\ChipBuffer.shp", "20 Miles")
arcpy.Exists(bufferResult)

In [ ]:
arcpy.GetCount_management(bufferResult)

## Use REST response with other Python modules

In [ ]:
geoJson = "http://services5.arcgis.com/fef1QOfOwB7ly9yl/arcgis/rest/services/ChiptolePlaces/FeatureServer/0/query?where=1=1&outFields=City,Sales&f=geojson"
noGeomGeoJson = "http://services5.arcgis.com/fef1QOfOwB7ly9yl/arcgis/rest/services/ChiptolePlaces/FeatureServer/0/query?where=1=1&outFields=City,Sales&returnGeometry=false&f=geojson"


In [ ]:
r = request(geoJson)
jsonResponse = urlopen(r, ''.encode("UTF-8"))
chipJson = json.loads(jsonResponse.read().decode('utf-8'))
from pandas.io.json import json_normalize
df = json_normalize(chipJson['features'])
print(df)

In [ ]:
r = request(noGeomGeoJson)
jsonResponse = urlopen(r, ''.encode("UTF-8"))
chipJson = json.loads(jsonResponse.read().decode('UTF-8'))
df = json_normalize(chipJson['features'])
print(df)

In [ ]:
#list columns
list(df.columns.values)

In [ ]:
#mean for sales
print(df.mean())

In [ ]:
#list stores by highest sales
print(df.sort(['properties.Sales'], ascending=[False]))

In [ ]:
#list bottom 5 stores:
print(df.sort(['properties.Sales'], ascending=True).head(5))

In [ ]:
#How much the bottom 5 stores brought in:
print(sum(df['properties.Sales'].order(ascending=True).head(5)))
#35754.281190542679

In [ ]:
#top 5 california cities
print(df[df['properties.City'].str.contains('CA')].sort(['properties.Sales'], ascending=False).head(5))